In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import confusion_matrix, classification_report

2023-12-17 12:40:30.695069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Funktion zum Einlesen einer Datei
def load_dat(path, input_end, crop_start, crop_stop):
    try:
        with open(path, 'r') as file:
            # Zeilen aus der Datei lesen und in eine Liste von Zeichenketten aufteilen
            lines = file.read().splitlines()

            # Die Daten in eine Matrix umwandeln
            data = [list(map(float, line.split()[:input_end])) for line in lines]
            data = np.array(data)

            # Format ermitteln und ggf. transponieren
            size = (len(data), len(data[0]))
            if (size[0] > 52):
                      data = np.transpose(data)
            
            data = data[crop_start:crop_stop]

        # Größe der neuen Matrix ausgeben
        #new_size = (len(data), len(data[0]))
        #print(f"Größe der Trainingsdaten Matrix: {new_size}")
        return data

    except FileNotFoundError:
        print(f'Die Datei {path} wurde nicht gefunden.')

    except Exception as e:
        print(f'Fehler beim Lesen der Datei: {str(e)}')


In [3]:
#RELPATH = 'Dataset\\'
RELPATH = '/home/ki-projekt/KI-Projekt/data/TennesseEastman/'

data_raw = []
dataTE_raw = []

# Dateien einlesen
for datei in sorted(os.listdir(RELPATH)):
    filePath = RELPATH + datei

    #print('Look, i found a file: ' + filePath)

    if 'te' in datei:
        data_matrix = load_dat(filePath, 480, 0, 480)
        dataTE_raw.append(data_matrix)
    else:
        data_matrix = load_dat(filePath, 480, 0, 480)
        data_raw.append(data_matrix)


data_raw = np.array(data_raw)
dataTE_raw = np.array(dataTE_raw)

In [4]:
# normieren
data_norm = []
dataTE_norm = []

for i in range(22):
    array2ndD = []
    array2ndDTE = []

    for k in range(52):
        normOfSensorData = np.linalg.norm(data_raw[i][k])
        normOfSensorDataTE = np.linalg.norm(dataTE_raw[i][k])
        meanOfSensorData = np.mean(data_raw[i][k])
        meanOfSensorDataTE = np.mean(dataTE_raw[i][k])
        
        array3rdD = []
        array3rdDTE = []
            
        for m in range(480):
            array3rdD.append((data_raw[i][k][m] - meanOfSensorData)/normOfSensorData)
            array3rdDTE.append((dataTE_raw[i][k][m] - meanOfSensorDataTE)/normOfSensorDataTE) ### clean up

        array2ndD.append(array3rdD)
        array2ndDTE.append(array3rdDTE)

    data_norm.append(array2ndD)
    dataTE_norm.append(array2ndDTE)

data_norm = np.array(data_norm)
dataTE_norm = np.array(dataTE_norm)


# Debug 
# print(np.max(data_norm))
# print(np.min(data_norm))
#print(data_norm)
#print(dataTE_norm.shape)

#print(data_raw[1][1].shape)
#print(data_raw[1][1])
#print(np.linalg.norm(data_raw[1], axis=1))

In [5]:
# aus normierter Matrix Trainigsdaten erstellen

training_data = []
test_data = []

for i in range(22):
      # hier label erzeugen -> one-hot vector
      oneHotVector = np.eye(22)[i, :]
      fault_data = np.transpose(data_norm[i])
      fault_dataTE = np.transpose(dataTE_norm[i])

      for k in range(480):
            training_data.append([fault_data[k], oneHotVector])
            test_data.append([fault_dataTE[k], oneHotVector])

# durchmischen

random.shuffle(training_data) # anpassen mit tf-Funktion mit batch-size etc.

In [6]:
### Training data
x_train = []
y_train = []

for testvalues, label in training_data:
      x_train.append(testvalues)
      y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)


In [7]:
#### Test data
x_test = []
y_test = []

for testvalues, label in test_data:
      x_test.append(testvalues)
      y_test.append(label)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
print(x_test.shape)
print(y_test.shape)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1], 1), return_sequences=True))

for _ in range(20):  # LSTM-Schichten hinzufügen
    model.add(LSTM(64, return_sequences=True))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(22, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modell trainieren
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Modellbewertung
train_accuracy = model.evaluate(x_train, y_train, verbose=0)[1]
test_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]

print(f'Training Accuracy: {train_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

# Confusion Matrix
y_pred = np.argmax(model.predict(x_test), axis=-1)
y_true = [i.argmax() for i in y_test]
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred)
print('Classification Report:')
print(class_report)